In [3]:
#####Latent Dirichlet Allocation Model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns

####データの発生####
##データの設定
k = 10   #トピック数
d = 2000   #文書数
v  = 400   #語彙数
w = poisson(200, d)   #1文書あたりの単語数
f = sum(w)   #総単語数

In [4]:
##パラメータの設定
#ディレクリ分布のパラメータの設定
alpha0 = np.repeat(0.3, k)   #文書のディレリ事前分布のパラメータ
alpha1 = np.repeat(0.25, v)   #単語のディレクリ事前分布のパラメータ

#ディレクリ分布からパラメータを発生
thetat = theta = np.random.dirichlet(alpha0, d)   #文書のトピック分布をディレクリ乱数から発生
phit = phi = np.random.dirichlet(alpha1, k)    #単語のトピック分布をデレクレリ乱数から発生

In [5]:
##多項分布より単語頻度行列を生成
WX = np.zeros((d, v), dtype='int')
Z = [i for i in range(d)]
vec = np.arange(0, k)

for i in range(d):
    #文書のトピックを生成
    z = multinomial(1, theta[i, :], w[i])   #文書のトピック分布を発生
    z_vec = np.dot(z, vec)

    #トピック割当から単語を生成
    word = np.zeros((w[i], v))
    for j in range(w[i]):
        word[j, :] = multinomial(1, phi[z_vec[j], :], 1)

    WX[i, :] = np.sum(word, axis=0)
    Z[i] = z_vec

In [6]:
####トピックモデルのためのデータと関数の準備####
##それぞれの文書中の単語の出現をベクトルに並べる
##データ推定用IDを作成
ID_list = [i for i in range(d)]
wd_list = [i for i in range(d)]

#文書ごとに求人IDおよび単語IDを作成
for i in range(d):
    ID_list[i] = np.repeat(i, w[i])
    num1 = (WX[i, :] > 0) * np.arange(1, v+1)
    num2 = num1[num1!=0]
    W1 = WX[i, WX[i, :] > 0]
    number = np.repeat(num2, W1)
    wd_list[i] = number

#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ID_d = np.zeros(sum(w), dtype='int')
start = 0

for i in range(d):
    wd[start:start+w[i]] = wd_list[i] - 1
    ID_d[start:start+w[i]] = ID_list[i]
    start += w[i] 

In [7]:
##インデックスを作成
doc_list = [i for i in range(d)]
word_list = [i for i in range(v)]
index = np.array(range(f))

for i in range(d):
    doc_list[i] = index[ID_d==i]
for i in range(v):
    word_list[i] = index[wd==i]

In [8]:
####マルコフ連鎖モンテカルロ法でLDAのパラメータをサンプリング####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w, k):
    Bur = np.zeros((f, k))
    Bur0 = np.zeros((f, k))

    #負担係数を計算
    for j in range(k):
        Bi = np.repeat(theta[:, j], w) * phi[j, wd]
        Bur[:, j] = Bi

    #負担率の分母部分
    Bur_sums = np.sum(Bur, axis=1)
    for l in range(k):
        Bur0[:, l] = Bur_sums 

    #負担率と混合率を計算
    Br = Bur / Bur0   #負担率の計算
    r = np.sum(Br, axis=0) / np.sum(Br)   #混合率の計算
    return Br, Bur, r

In [9]:
##アルゴリズムの設定
R = 10000
keep = 2
iter = 0 
disp =10

In [10]:
##事前分布の設定
alpha1 = 1.0 
beta1 = 0.5

In [11]:
##パラメータの初期値
mu = np.sum(WX, axis=0)/np.sum(WX)
phi = np.random.dirichlet(np.repeat(mu*100, v), k)
theta = np.random.dirichlet(np.repeat(5, k), d)

In [12]:
##パラメータの保存用配列
THETA = np.zeros((d, k, R//keep), dtype='int')
PHI = np.zeros((k, v, R//keep), dtype='int')
SEG = np.zeros((f, k), dtype='int')

In [16]:
####ギブスサンプリングでパラメータをサンプリング####
out = burden_fr(f, theta, phi, wd, w, k)
Br = out[0]
r = out[2]

In [18]:
for i in range(f):
    multinomial(1, Br[i, :], 1)

In [131]:
word_cumsum = np.cumsum(Br, axis=0)
 = np.random.rand(f)

array([ 0.98645436,  0.22496953,  0.23050976, ...,  0.53068299,
        0.98614637,  0.87151505])

In [144]:
np.matlib.repmat(np.random.rand(f), k, 1).shape

(10, 400582)

In [24]:
'abcde'[0:3]

'abc'